## Capstone

For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [31]:
# Grab the data from here: http://share.mailcharts.com/0D0Q2e0L1s47 and http://share.mailcharts.com/0z0F3m1X0l39
import pandas as pd
import numpy as np

In [32]:
df = pd.read_csv("./data/capstone-v2.csv", encoding="ISO-8859-1")
print(df.shape)
print(df.columns)

(3365, 8)
Index(['reg_id', 'add_id', 'name', 'email_guid', 'sent_at', 'subject',
       'full_text', 'r'],
      dtype='object')


In [33]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import ngrams
import string
import re

# Steps: Clean up text, stemming, remove stop words and weird chars, tokenizer words

# punctuation = list(set(string.punctuation))
re_punctuation = "\.|\>|\/|\)|\"|\(|\}|\'|\_|\-|\$|\:|\[|\^|\+|\?|\`|\~|\!|\<|\@|\;|\=|\*|\\\|\{|\&|\]|\||\,|\|"
stopwords_set = list(set(stopwords.words('english')))
handpicked_works = ["com"]

def get_unigram_sentence(sentence, company_name):
    company_names = company_name.lower().split(" ")
    company_names.append(company_name.lower().replace(" ", ""))
    
    sentence_no_punc = re.sub(re_punctuation, " ", sentence)
    unigram = [word for word in word_tokenize(sentence_no_punc.lower()) if word not in stopwords_set and word not in company_names and word not in handpicked_works]
    return unigram

In [34]:
tokenized_text = []

for i, el in df.iterrows():
    tokenized_text.append(get_unigram_sentence(el['subject'], el['name']))

df["tokenized_text"] = tokenized_text

In [35]:
stemmer = PorterStemmer()

def get_stems(words):
    return [stemmer.stem(word) for word in words]

In [36]:
df["stemmed_tokens"] = df.tokenized_text.apply(lambda x: get_stems(x))

In [37]:
df["stemmed_text"] = df["stemmed_tokens"].apply(lambda x: " ".join(word for word in x))

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# vectorizer = TfidfVectorizer(stop_words="english", min_df=50, ngram_range=(1,2))
vectorizer = CountVectorizer(stop_words="english", min_df=50)

vectorizer = vectorizer.fit(df["stemmed_text"])
X = vectorizer.transform(df["stemmed_text"])
X = X.toarray()

In [39]:
# # Split our data in test / train
# from sklearn.model_selection import train_test_split

# X_train, X_test = train_test_split(X, test_size=0.25, random_state=100)
# df_train, df_test = train_test_split(df, test_size=0.25, random_state=100)

# print(X_train.shape)
# print(X_test.shape)

In [40]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

vectorized_values = X

lda = LDA(6, random_state=22)
lda.fit(vectorized_values)

//anaconda/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=6, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=22,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [41]:
labels = []

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]])
        labels.append([(vectorizer.get_feature_names()[i]) for i in topic.argsort()[:-1-1:-1]][0])
        print("=" * 100)

print_topics(lda, vectorizer)

Topic 0:
[('order', 482.15211872257055), ('confirm', 255.62581284201485), ('account', 95.85691550403682), ('receiv', 49.904849740078475), ('lorem', 0.1678661424038993), ('shipment', 0.1673806752796114), ('deliveri', 0.16722944819457455), ('shop', 0.16713224124860648), ('ship', 0.16705559786669114), ('thank', 0.1669234619469451)]
Topic 1:
[('thank', 209.84719430827846), ('order', 155.5153540726883), ('lorem', 104.28456241599727), ('gift', 97.781533536753), ('way', 87.70228529196665), ('shipment', 53.35132204199367), ('shop', 0.16766427225987832), ('welcom', 0.1672196995100544), ('account', 0.16721608074336233), ('purchas', 0.16712906671594271)]
Topic 2:
[('ship', 314.80376502297815), ('welcom', 299.09929360261737), ('free', 163.94775339994263), ('order', 163.45352267114652), ('today', 98.55312978529751), ('lorem', 0.17229374873845626), ('start', 0.16898496289656215), ('end', 0.16845898481465585), ('10', 0.16768290883576606), ('day', 0.16764003951902143)]
Topic 3:
[('sale', 96.4762128554

In [42]:
# Next, let's assign this to our original text

vectorized_values_lda = lda.transform(vectorized_values)

predicted_label = []
for i in vectorized_values_lda:
    # Get the highest value. We'll consider that to be the predicted label.
    predicted_label.append(labels[i.argsort()[-1]])

df["lda_predicted_label"] = predicted_label

In [43]:
df.head()

,reg_id,add_id,name,email_guid,sent_at,subject,full_text,r,tokenized_text,stemmed_tokens,stemmed_text,lda_predicted_label
0,6361,7526,Le Creuset,45f2d9ed-128e-9ae5-b8f1-4e224a02dfca,2017-01-10 21:34:33,"Welcome, Lorem Ipsum!",LE CREUSET Welcome to Le Creuset. To log in wh...,1,"[welcome, lorem, ipsum]","[welcom, lorem, ipsum]",welcom lorem ipsum,ship
1,6361,7526,Le Creuset,34db5cee-2a9c-17f4-b97d-68343ad26f19,2017-01-10 21:36:48,"Hi! You were looking for free shipping, right?",Save a bundle on shipping with code LECREUSETL...,2,"[hi, looking, free, shipping, right]","[hi, look, free, ship, right]",hi look free ship right,ship
2,6361,7526,Le Creuset,55f96ec8-739f-4a3a-63c4-ec1fddcf795d,2017-01-10 21:41:43,Le Creuset: New Order # 200068673,LE CREUSET Thank you for your order from Le Cr...,3,"[new, order, #, 200068673]","[new, order, #, 200068673]",new order # 200068673,new
3,6361,7526,Le Creuset,f0188d30-aa26-8614-7a44-aa149fad66b0,2017-01-12 21:37:00,Your kitchen + our color choices = food heaven,What will you bring to the table? View in brow...,4,"[kitchen, color, choices, food, heaven]","[kitchen, color, choic, food, heaven]",kitchen color choic food heaven,day
4,6361,7526,Le Creuset,29af73ae-8956-9bdc-7148-d5eb0bde173b,2017-01-13 15:09:04,Free Shipping Starts Now + Storage Staples to ...,"Plus, a sweet treat for you! LE CREUSET Cookwa...",5,"[free, shipping, starts, storage, staples, get...","[free, ship, start, storag, stapl, get, organ]",free ship start storag stapl get organ,ship


In [44]:
# Compare against other classifiers

from sklearn.decomposition import NMF
from sklearn.cluster import KMeans as KM 
from sklearn.cluster import AgglomerativeClustering as AG

In [55]:
_model = NMF(6, random_state=22)
_model.fit(vectorized_values)

labels = []
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]])
        labels.append([(vectorizer.get_feature_names()[i]) for i in topic.argsort()[:-1-1:-1]][0])
        print("=" * 100)
print_topics(_model, vectorizer)

vectorized_values_model = _model.transform(vectorized_values)

predicted_label = []
for i in vectorized_values_model:
    # Get the highest value. We'll consider that to be the predicted label.
    predicted_label.append(labels[i.argsort()[-1]])

df["nmf_predicted_label"] = predicted_label

Topic 0:
[('order', 5.651839590055607), ('ship', 0.4213161251264974), ('way', 0.4196380610541833), ('receiv', 0.2997929911071086), ('shipment', 0.18355887601843895), ('10', 0.0403437508983332), ('deliveri', 0.031041842306667945), ('thank', 0.02682125866570286), ('15', 0.0031899734948774862), ('confirm', 0.000222588516833546)]
Topic 1:
[('ship', 3.713852826053389), ('free', 2.5552352981043587), ('day', 0.7835126859541285), ('50', 0.5285927523917192), ('today', 0.44003367256027215), ('save', 0.4047563294996324), ('end', 0.39632438860590685), ('sale', 0.3789483767954335), ('20', 0.35191955119013246), ('10', 0.3149877678428647)]
Topic 2:
[('welcom', 4.09750534581979), ('lorem', 0.5457570371041985), ('15', 0.27051541654673017), ('10', 0.20263742277178315), ('gift', 0.20247788277932735), ('account', 0.10861971464294792), ('save', 0.10259128624557853), ('20', 0.08463227650315443), ('start', 0.07658995275088007), ('purchas', 0.04195397791366289)]
Topic 3:
[('new', 3.930216978389036), ('sale', 

In [145]:
_model = KM(6, random_state=22)
_model.fit(vectorized_values)

labels = []
def print_topics(model, vectorizer, top_n=3):
    for i, t in enumerate(_model.cluster_centers_):
        top_words = t.argsort()[:-3:-1]
#         for w in top_words:
#             print(vectorizer.get_feature_names()[w])
        labels.append(vectorizer.get_feature_names()[top_words[0]])
#         print("=" * 100)
print_topics(_model, vectorizer)

vectorized_values_model = _model.transform(vectorized_values)

predicted_label = []
for i in vectorized_values_model:
    # Get the highest value. We'll consider that to be the predicted label.
    predicted_label.append(labels[i.argsort()[-1]])

label = model_names[m] + "_predicted_label"
df["km_predicted_label"] = predicted_label

In [147]:
df

,reg_id,add_id,name,email_guid,sent_at,subject,full_text,r,tokenized_text,stemmed_tokens,stemmed_text,lda_predicted_label,nmf_predicted_label,km_predicted_label
0,6361,7526,Le Creuset,45f2d9ed-128e-9ae5-b8f1-4e224a02dfca,2017-01-10 21:34:33,"Welcome, Lorem Ipsum!",LE CREUSET Welcome to Le Creuset. To log in wh...,1,"[welcome, lorem, ipsum]","[welcom, lorem, ipsum]",welcom lorem ipsum,ship,welcom,order
1,6361,7526,Le Creuset,34db5cee-2a9c-17f4-b97d-68343ad26f19,2017-01-10 21:36:48,"Hi! You were looking for free shipping, right?",Save a bundle on shipping with code LECREUSETL...,2,"[hi, looking, free, shipping, right]","[hi, look, free, ship, right]",hi look free ship right,ship,ship,order
2,6361,7526,Le Creuset,55f96ec8-739f-4a3a-63c4-ec1fddcf795d,2017-01-10 21:41:43,Le Creuset: New Order # 200068673,LE CREUSET Thank you for your order from Le Cr...,3,"[new, order, #, 200068673]","[new, order, #, 200068673]",new order # 200068673,new,new,free
3,6361,7526,Le Creuset,f0188d30-aa26-8614-7a44-aa149fad66b0,2017-01-12 21:37:00,Your kitchen + our color choices = food heaven,What will you bring to the table? View in brow...,4,"[kitchen, color, choices, food, heaven]","[kitchen, color, choic, food, heaven]",kitchen color choic food heaven,day,confirm,order
4,6361,7526,Le Creuset,29af73ae-8956-9bdc-7148-d5eb0bde173b,2017-01-13 15:09:04,Free Shipping Starts Now + Storage Staples to ...,"Plus, a sweet treat for you! LE CREUSET Cookwa...",5,"[free, shipping, starts, storage, staples, get...","[free, ship, start, storag, stapl, get, organ]",free ship start storag stapl get organ,ship,ship,order
5,6361,7526,Le Creuset,f6a07758-4be1-2c3a-4c88-83dc8fb68712,2017-01-13 23:59:42,Le Creuset: Shipment # 200064834 for Order # 2...,LE CREUSET Thank you for your order from Le Cr...,6,"[shipment, #, 200064834, order, #, 200068673]","[shipment, #, 200064834, order, #, 200068673]",shipment # 200064834 order # 200068673,thank,order,free
6,6361,7526,Le Creuset,9fdbe5dc-7d44-b243-c97e-1bd8301c0662,2017-01-15 15:09:46,Make it a Joyful Year,"To view this email as a web page, go here. LE ...",7,"[make, joyful, year]","[make, joy, year]",make joy year,day,confirm,order
7,6361,7526,Le Creuset,20bcf82b-2c94-94ac-71ea-8e0fc6513f04,2017-01-15 21:37:01,Customize your cookware and your inbox,Fill out your preferences to get emails just f...,8,"[customize, cookware, inbox]","[custom, cookwar, inbox]",custom cookwar inbox,day,confirm,order
8,6361,7526,Le Creuset,1cd9bf8a-8506-5b9d-df34-ed30c5713de1,2017-01-16 15:07:35,Free Shipping Ends Today + Reasons to Love Our...,Get inspired to cook at home this month. LE CR...,9,"[free, shipping, ends, today, reasons, love, o...","[free, ship, end, today, reason, love, oval, o...",free ship end today reason love oval oven,ship,ship,order
9,6361,7526,Le Creuset,6425ce6f-e013-f350-62e2-53593752f52b,2017-01-18 15:07:08,Introducing the Le Creuset Gift Registry,Register now or shop our top wedding gifts. LE...,10,"[introducing, gift, registry]","[introduc, gift, registri]",introduc gift registri,thank,welcom,order


In [ ]:
# Add AG model

ag = AG(6)
ag.fit(vectorized_values)

labels = []
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]])
        labels.append([(vectorizer.get_feature_names()[i]) for i in topic.argsort()[:-1-1:-1]][0])
        print("=" * 100)
print_topics(ag, vectorizer)

vectorized_values_model = _model.transform(vectorized_values)

predicted_label = []
for i in vectorized_values_model:
    # Get the highest value. We'll consider that to be the predicted label.
    predicted_label.append(labels[i.argsort()[-1]])

label = model_names[m] + "_predicted_label"
df[label] = predicted_label

In [18]:
# Load manual classification
classified = pd.read_csv("./data/captsone-manually-classified.csv")

# Fill blanks with 0
classified.fillna(0, inplace=True)

# Remove any "?" and replace with 0
classified["order confirmation"][classified["order confirmation"] == "?"] = 0

# Print some quick summary stats
print(classified["order confirmation"].value_counts())
print(classified["shipping confirmation"].value_counts())
print(classified["delivery notification"].value_counts())

0    1717
1     206
Name: order confirmation, dtype: int64
0.0    1769
1.0     154
Name: shipping confirmation, dtype: int64
0.0    1906
1.0      17
Name: delivery notification, dtype: int64


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [19]:
def get_classification_results(row):
    if row["order confirmation"] != 0:
        return "order"
    if row["shipping confirmation"] != 0:
        return "ship"
    if row["delivery notification"] != 0:
        return "delivery"
    else:
        return "not classified"

classified["classification"] = classified.apply(get_classification_results, axis=1)
# classified.head(100)

In [20]:
# Merge our manual classification with the main DF
df = df.merge(classified[["email_guid", "classification"]], on="email_guid")

In [22]:
df.classification.value_counts()

not classified    1556
order              209
ship               160
delivery            17
Name: classification, dtype: int64

In [23]:
df.head()

,reg_id,add_id,name,email_guid,sent_at,subject,full_text,r,tokenized_text,stemmed_tokens,stemmed_text,lda_predicted_label,nmf_predicted_label,km_predicted_label,classification
0,6361,7526,Le Creuset,45f2d9ed-128e-9ae5-b8f1-4e224a02dfca,2017-01-10 21:34:33,"Welcome, Lorem Ipsum!",LE CREUSET Welcome to Le Creuset. To log in wh...,1,"[welcome, lorem, ipsum]","[welcom, lorem, ipsum]",welcom lorem ipsum,ship,ship,ship,not classified
1,6361,7526,Le Creuset,34db5cee-2a9c-17f4-b97d-68343ad26f19,2017-01-10 21:36:48,"Hi! You were looking for free shipping, right?",Save a bundle on shipping with code LECREUSETL...,2,"[hi, looking, free, shipping, right]","[hi, look, free, ship, right]",hi look free ship right,ship,thank,ship,not classified
2,6361,7526,Le Creuset,55f96ec8-739f-4a3a-63c4-ec1fddcf795d,2017-01-10 21:41:43,Le Creuset: New Order # 200068673,LE CREUSET Thank you for your order from Le Cr...,3,"[new, order, #, 200068673]","[new, order, #, 200068673]",new order # 200068673,new,sale,sale,order
3,6361,7526,Le Creuset,f0188d30-aa26-8614-7a44-aa149fad66b0,2017-01-12 21:37:00,Your kitchen + our color choices = food heaven,What will you bring to the table? View in brow...,4,"[kitchen, color, choices, food, heaven]","[kitchen, color, choic, food, heaven]",kitchen color choic food heaven,day,day,ship,not classified
4,6361,7526,Le Creuset,29af73ae-8956-9bdc-7148-d5eb0bde173b,2017-01-13 15:09:04,Free Shipping Starts Now + Storage Staples to ...,"Plus, a sweet treat for you! LE CREUSET Cookwa...",5,"[free, shipping, starts, storage, staples, get...","[free, ship, start, storag, stapl, get, organ]",free ship start storag stapl get organ,ship,thank,ship,not classified


In [ ]:
# Next step: Use logistic regression to try to predict an email's label using our unsupervised model